In [9]:
import pandas as pd

from createDataset import load_tracks
from dataclasses import asdict

In [ ]:
tracks = load_tracks()

In [10]:
df = pd.DataFrame.from_records([asdict(x) for x in tracks])
df

,id,artist_id,album_id,durationInSec,genres,instruments,moods,gdrive_nr,gdrive_id,gdrive_size_in_gb,main_genre,main_instrument,main_mood,chunk_nr,mp3_sha256
0,1142857,429523,137046,40.5,[],[],"[game, holiday, children]",57,16IC_mRlaYGClWtK0xTgVcDacZGiUh6J4,5.6,,,children,0,e0ce88bf52ca1e80f18fa2df852dd9bd445285bd331a82...
1,973747,375104,114654,43.7,[],[],[christmas],47,1YqfP6cNHWfTJ5qOS9VpMIbKYM9bMRUk5,5.5,,,christmas,0,a3515d5e3542fdfa5dc4d08a6624ce0717c431dfb1603b...
2,973751,375104,114654,46.7,[],[],[christmas],51,1qbPA5xI8dd-5Jmo8YLL-uxQPChp_GNb8,5.8,,,christmas,0,17ecae99bde9dcc1528fded29b54e4de013c126d89a969...
3,973752,375104,114654,125.0,[],[],[christmas],52,1zOi_taXSNa6pcZ-4dKDaLLAc6957C5UT,5.5,,,christmas,0,502d1091563399c7676b8e2108fe9ecf7cf1de52a788fe...
4,973753,375104,114654,152.7,[],[],[christmas],53,1B3hd095S2Kw8ZYSjk7kp9UJMvLgChWKz,5.5,,,christmas,0,96d2e7c4319382278d5455389495324d244cba8636cd65...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55696,1088053,375935,129718,140.0,"[ethnicrock, world]","[violin, tabla, sitar]","[travel, documentary, india]",53,1B3hd095S2Kw8ZYSjk7kp9UJMvLgChWKz,5.5,world,violin,documentary,239,19307e73afaacdfd56c4dac6fe9ab511cb801655b5bf98...
55697,1137237,439567,136587,192.7,[world],"[voice, flute]",[],37,1Na9vgTCUvrF3Bg8h4FrYs4jRzAFuCigB,5.3,world,voice,,239,34bd631e8996945a19ae481b2841239a506d4aef8ac584...
55698,1124726,432273,134534,128.6,"[world, jungle, tribal]","[tambourine, pandeiro, glockenspiel, vibraphon...","[videogame, children, funny]",26,19REUDBP2N-C0Dy5TGdqlPGNsC63zYEIR,5.2,world,voice,children,239,5e2bc1d5129dbf38b59ba3baf0527c041408d679ce22ba...
55699,918148,375935,108714,165.1,"[ethnicrock, african, world]","[bongo, conga, voice]","[travel, brazil]",48,1Xuc_K5DwrOsHQbB-LDUwUExcGNMpJu2f,5.6,world,voice,travel,239,d4dbd73ff140051cc411a6a4d0cd00a62b41df5b6a7e7d...


In [11]:
import numpy as np

val_ration = 0.1
df["split"] = ["val" if x else "train" for x in np.random.rand(len(df)) < val_ration]

In [13]:
df["split"].value_counts()

train    49982
val       5719
Name: split, dtype: int64

In [21]:
t = df.explode("genres").groupby(["split", "genres"]).agg({"id": "count", "durationInSec": "sum"}).unstack("split").sort_values(("id", "train"), ascending=0)
t["val_ratio"] = t[("id", "val")] / (t[("id", "val")]+t[("id", "train")])
t["val_duration"] = t[("durationInSec", "val")] / (t[("durationInSec", "val")] + t[("durationInSec", "train")])
t

id         durationInSec           val_ratio val_duration
split             train     val         train       val                       
genres                                                                        
electronic      14648.0  1678.0     3941082.3  442683.6  0.102781     0.100982
soundtrack       7108.0   838.0     1471447.6  175012.1  0.105462     0.106296
pop              6986.0   819.0     1555686.0  180176.2  0.104933     0.103796
ambient          6680.0   787.0     1971768.5  218944.2  0.105397     0.099942
rock             6130.0   735.0     1419177.5  171545.9  0.107065     0.107841
...                 ...     ...           ...       ...       ...          ...
abstracthiphop      7.0     NaN        1753.8       NaN       NaN          NaN
emocore             5.0     NaN        1365.0       NaN       NaN          NaN
christianrap        4.0     NaN         728.7       NaN       NaN          NaN
droneambient        2.0     1.0         551.1     351.6  0.333333     0.389498
deutschrock         1.0     NaN         249.9       NaN       NaN          NaN

[252 rows x 6 columns]

In [23]:
df.to_parquet("tracks.parquet")